In [0]:

-- filled_value null 추가 보완 : '24/09 이전 로그 탐색
with extension_dpv_webos22 as (
  
  -- 마지막 dpv 가져오기 : 
  select mac_addr, X_Device_SDK_VERSION as dpv
  from   (
      select mac_addr, row_number() over(partition by mac_addr order by log_create_time desc) as rn, X_Device_SDK_VERSION
      from   eic_data_ods.tlamp_private.normal_log_webos22
      where  date_ym >= '2022-03'
        AND  date_ym < '2024-09'
        AND  context_name = 'tvpowerd'
        AND  message_id   = 'NL_POWER_STATE'
        AND  mac_addr in (
          select distinct mac_addr
          from   sandbox.z_eunmi1_ko.temp_normal_log_webos22
          where  filled_dpv is null 
      ) 
  ) 
  where rn = 1
), extension_dpv_webos23 as (
  
  -- 마지막 dpv 가져오기 : 
  select mac_addr, X_Device_SDK_VERSION as dpv
  from   (
      select mac_addr, row_number() over(partition by mac_addr order by log_create_time desc) as rn, X_Device_SDK_VERSION
      from   eic_data_ods.tlamp_private.normal_log_webos23
      where  date_ym >= '2023-03'
        AND  date_ym < '2024-09'
        AND  context_name = 'tvpowerd'
        AND  message_id   = 'NL_POWER_STATE'
        AND  mac_addr in (
          select distinct mac_addr
          from   sandbox.z_eunmi1_ko.temp_normal_log_webos23
          where  filled_dpv is null 
      ) 
  ) 
  where rn = 1
), extension_dpv_webos24 as (
  
  -- 마지막 dpv 가져오기 : 
  select mac_addr, X_Device_SDK_VERSION as dpv
  from   (
      select mac_addr, row_number() over(partition by mac_addr order by log_create_time desc) as rn, X_Device_SDK_VERSION
      from   eic_data_ods.tlamp_private.normal_log_webos24
      where  date_ym >= '2024-03'
        AND  date_ym < '2024-09'
        AND  context_name = 'tvpowerd'
        AND  message_id   = 'NL_POWER_STATE'
        AND  mac_addr in (
          select distinct mac_addr
          from   sandbox.z_eunmi1_ko.temp_normal_log_webos24
          where  filled_dpv is null 
      ) 
  ) 
  where rn = 1
), mart_final_webos22 as (
  select mart.mac_addr, mart.dim_date_ym, mart.country_code, mart.platform_version, mart.platform_code, mart.sales_model_code, mart.inch
        , (case when mart.filled_dpv is null then ext.dpv else mart.filled_dpv end) as final_dpv
  from   sandbox.z_eunmi1_ko.temp_normal_log_webos22 as mart
  left join extension_dpv_webos22 as ext using (mac_addr)
), mart_final_webos23 as (
  select mart.mac_addr, mart.dim_date_ym, mart.country_code, mart.platform_version, mart.platform_code, mart.sales_model_code, mart.inch
        , (case when mart.filled_dpv is null then ext.dpv else mart.filled_dpv end) as final_dpv
  from   sandbox.z_eunmi1_ko.temp_normal_log_webos23 as mart
  left join extension_dpv_webos23 as ext using (mac_addr)
), mart_final_webos24 as (
  select mart.mac_addr, mart.dim_date_ym, mart.country_code, mart.platform_version, mart.platform_code, mart.sales_model_code, mart.inch
        , (case when mart.filled_dpv is null then ext.dpv else mart.filled_dpv end) as final_dpv
  from   sandbox.z_eunmi1_ko.temp_normal_log_webos24 as mart
  left join extension_dpv_webos24 as ext using (mac_addr)
)

  -- 지표 결과 2 : 16,722 return
select tr.*, tc.region
from  (
  select dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv, count(distinct mac_addr) as cnt_ud
  from   mart_final_webos22
  where  final_dpv is not null
  group by dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv

  union all 
  select dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv, count(distinct mac_addr) as cnt_ud
  from   mart_final_webos23
  where  final_dpv is not null
  group by dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv

  union all 
  select dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv, count(distinct mac_addr) as cnt_ud
  from   mart_final_webos24
  where  final_dpv is not null
  group by dim_date_ym, country_code, platform_version, platform_code, sales_model_code, inch, final_dpv
) tr
left join (
  select distinct country_code AS cc, region
  from eic_data_dimension.common.country_code
  where region = 'EIC'
) as tc ON lower(tr.country_code) = lower(tc.cc)
where tc.cc is not null and len(replace(final_dpv, ' ', '')) > 1





Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.